# 🎯 Complex Troubleshooting: The Orchestrator-Workers Pattern

Welcome to orchestrator-workers! This pattern shines when:
- Tasks need dynamic subtask planning
- You can't predict all steps in advance
- Work needs coordinated execution

The main difference between an orchestrator and a parallelizer is that we don't know the steps to take ahead of time. Instead we'll use an LLM to dynamically assign tasks. 

Perfect for complex OpenSearch troubleshooting! 🚀

Lets start with setting up our clients and retrieval. 

In [ ]:
import boto3

from utils.retrieval_utils import get_chroma_os_docs_collection, ChromaDBRetrievalClient

# Initialize the Bedrock client
REGION = 'us-west-2'
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime', region_name=REGION)

# We've pushed the retrieval client from the prompt chaining notebook to the retrieval utils for simplicity
chroma_os_docs_collection: ChromaDBRetrievalClient = get_chroma_os_docs_collection()

print("✅ Client setup and retrieval client complete!")

# Create Helpers
Reuse the same helpers from our previous workshops

In [ ]:
from typing import Type, Dict, Any, List

# We pushed the base propmt from the previous lab to a a base prompt file.
from utils.base_prompt import BasePrompt
from utils.retrieval_utils import RetrievalResult

def call_bedrock(prompt: BasePrompt) -> str:
    kwargs = {
        "modelId": prompt.model_id,
        "inferenceConfig": prompt.hyperparams,
        "messages": prompt.to_bedrock_messages(),
        "system": prompt.to_bedrock_system(),
    }

    # Call Bedrock
    converse_response: Dict[str, Any] = bedrock.converse(**kwargs)
    # Get the model's text response
    return converse_response['output']['message']['content'][0]['text']

# Helper function to call bedrock
def do_rag(user_input: str, rag_prompt: Type[BasePrompt]) -> str:
    # Retrieve the context from the vector store
    retrieval_results: List[RetrievalResult] = chroma_os_docs_collection.retrieve(user_input, n_results=2)
    # Format the context into a string
    context: str = "\n\n".join([result.document for result in retrieval_results])

    print("Retrieval done")
    # Create the RAG prompt
    inputs: Dict[str, Any] = {"question": user_input, "context": context}
    rag_prompt: BasePrompt = rag_prompt(inputs=inputs)
    # Call Bedrock with the RAG prompt

    print("Calling Bedrock")
    return call_bedrock(rag_prompt)

## 1. Creating Our Troubleshooting System

We'll build a system with:
1. An orchestrator that plans diagnostic steps
2. Workers that handle specific tasks:
   - Issue identification
   - Test suggestion
   - Resolution steps

First lets create our prompts

In [ ]:
from typing import List, Dict, TypedDict, Annotated
import operator

from utils.base_prompt import BasePrompt

# Define troubleshooting prompts inheriting from BasePrompt
class PlanningPrompt(BasePrompt):
    system_prompt: str = "You are an expert OpenSearch diagnostician. Your role is to identify potential causes for issues."
    user_prompt: str = """
    Plan the diagnostic steps needed for this OpenSearch issue:
    {problem}
    
    Analyze the issue and return a list of specific potential problems 
    that could be causing this issue. Be specific and thorough.
    
    For example, instead of just "configuration issues", specify "incorrect 
    shard allocation settings" or "memory allocation problems".
    
    Return each potential issue as a separate line. At most return 3 potential issues.
    """

###########################################################################################
# Notice how we're using Amazon Nova Micro on the investor tasks
# Micro is extremely fast so we can generate lots of content in a short amount of time. 
###########################################################################################

class InvestigationPrompt(BasePrompt):
    model_id: str = "us.amazon.nova-micro-v1:0"
    system_prompt: str = "You are an expert OpenSearch troubleshooter. Provide detailed diagnostic and resolution information."
    user_prompt: str = """
    Regarding OpenSearch problem: 
    {question}
    
    Here's the context provided for the problem.
    {context}
    
    Explain how to diagnose if this is the actual problem and how to fix it.
    Include:
    1. Diagnostic commands or API calls
    2. Expected symptoms if this is the issue
    3. Step-by-step resolution steps
    4. Preventive measures
    """

class SynthesisPrompt(BasePrompt):
    system_prompt: str = "You are an expert OpenSearch engineer. Create a comprehensive, well-structured troubleshooting report."
    user_prompt: str = """
    Create a comprehensive troubleshooting report for this OpenSearch issue:
    {problem}
    
    Here are the findings from our investigation of potential causes:
    
    {issues_summary}
    
    Synthesize these findings into:
    1. Most likely root causes (ranked)
    2. Complete diagnostic steps
    3. Recommended resolution approach
    4. Verification steps to confirm the fix worked
    """

Next lets define our states. LangGraph handles dynamic worker creation with two separate states. The planner can use a worker with a separate state and send N number of tasks to be executed. 

Lastly, it uses the operator.add. This creates an aggregate of the values from the state dict so that we don't have race conditions where different threads are trying to overwrite each other. 

In [ ]:
from typing import TypedDict, List, Annotated, Dict
import operator

# Define the state structure to track our workflow
class TroubleshootingState(TypedDict):
    problem: str
    diagnostic_plan: List[str]
    investigation_results: Annotated[List[Dict[str, str]], operator.add]  # For parallel workers to add results
    final_report: str

# Define worker state
class WorkerState(TypedDict):
    problem: str
    issue: str
    investigation_results: Annotated[List[Dict[str, str]], operator.add]

In [ ]:
from langgraph.graph import StateGraph, END, START
from langgraph.constants import Send

def plan_diagnostics(state: TroubleshootingState) -> TroubleshootingState:
    """Orchestrator: Plans the troubleshooting steps needed"""
    
    # Create the planning prompt with proper inputs
    planning_prompt = PlanningPrompt(inputs={"problem": state["problem"]})
    
    # Call bedrock using the planning prompt
    plan = call_bedrock(planning_prompt)
    
    # Extract each diagnostic step as a potential issue to investigate
    steps = [step.strip() for step in plan.split('\n') if step.strip()]
    
    # Return updated state with diagnostic plan
    return {
        **state,
        "diagnostic_plan": steps,
        "investigation_results": []  # Initialize empty list for results
    }

def investigate_issue(state: WorkerState) -> WorkerState:
    """Worker: Uses RAG to investigate a specific potential issue"""
    
    # Create a more focused query for the RAG system
    search_query = f"OpenSearch {state['problem']} {state['issue']}"
    
    # Use the do_rag helper function with the proper prompt class and inputs
    investigation_result = do_rag(
        search_query,  # Use this as the search query for RAG
        InvestigationPrompt
    )
    
    # Return a list with a single result dictionary to be added to the main results
    return {
        "investigation_results": [{"issue": state["issue"], "result": investigation_result}]
    }

def synthesize_findings(state: TroubleshootingState) -> TroubleshootingState:
    """Synthesizer: Creates a unified diagnostic response"""
    
    # Format all investigated issues into a structured summary
    issues_summary = ""
    for result in state["investigation_results"]:
        issues_summary += f"\n## Issue: {result['issue']}\n{result['result']}\n"
    
    # Create the synthesis prompt with proper inputs
    synthesis_prompt = SynthesisPrompt(inputs={
        "problem": state["problem"],
        "issues_summary": issues_summary
    })
    
    # Generate the unified response
    final_report = call_bedrock(synthesis_prompt)
    
    # Return the updated state with the final report
    return {
        **state,
        "final_report": final_report
    }

# Create worker assignments using the Send API
def assign_workers(state: TroubleshootingState):
    """Assign a worker to each diagnostic issue in parallel"""
    
    # Create a Send message for each issue identified by the planner
    return [
        Send("investigate_issue", { "problem": state["problem"], "issue": issue }) 
        for issue in state["diagnostic_plan"]
    ]



def create_troubleshooting_workflow():
    """Creates a parallel workflow for orchestrated troubleshooting using RAG"""
    # Initialize the state graph with our state structure
    workflow = StateGraph(TroubleshootingState)
    
    # Add nodes to our graph
    workflow.add_node("plan", plan_diagnostics)
    workflow.add_node("investigate_issue", investigate_issue)
    workflow.add_node("synthesize", synthesize_findings)
    
    # Connect the workflow
    workflow.add_edge(START, "plan")
    workflow.add_conditional_edges("plan", assign_workers, ["investigate_issue"])
    workflow.add_edge("investigate_issue", "synthesize")
    workflow.add_edge("synthesize", END)
    
    return workflow.compile()

# Create and use the workflow
troubleshooter = create_troubleshooting_workflow()

# Execute our troubleshooter
Lastly, lets pick a question and run our orchestrator against it. 

In [ ]:
# Initialize with our test problem
initial_state = {
    "problem": "OpenSearch cluster not responding to queries, showing red health status",
    "diagnostic_plan": [],
    "investigation_results": [],
    "final_report": ""
}

# Run the troubleshooting workflow
result = troubleshooter.invoke(initial_state)

print("🔍 Troubleshooting Report Generated\n")
print(result["final_report"])

## 3. Benefits of the Orchestrator-Workers Pattern

In this lab we created an orchestrator that can dynamically assign tasks and delegate to workers in parallel.

Our troubleshooting system provides several advantages:

✅ Dynamic planning based on the specific problem

✅ Workers allocated for each task

✅ Coordinated problem-solving

✅ Comprehensive troubleshooting reports

Next, we'll explore how to improve our answers using the evaluator-optimizer pattern! 🚀